In [5]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

bc_data = datasets.load_breast_cancer()
df = pd.DataFrame(data= np.c_[bc_data['data'], bc_data['target']],
                     columns=list(bc_data['feature_names']) + ['target'])

X = df.values[:, :-1]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


pipeline = Pipeline([
    ('scl', StandardScaler()),
    ('pca', PCA()),
    ('lr', LogisticRegression())])
pipeline.fit(X_train, y_train)

print('Test acc: {}'.format(pipeline.score(X_test, y_test)))


Test acc: 0.982456140351
